## 1. Configuración e Imports

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Agregar src al path
project_dir = Path.cwd().parent
if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))

# Agregar src al path explícitamente para imports internos
src_dir = project_dir / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

from config import RANDOM_SEED
from src.data_loading import load_raw_training_data, load_raw_test_data, save_processed_data
from src.preprocessing import (
    clean_text, 
    extract_urls, 
    extract_mentions, 
    extract_hashtags,
    detect_intensified_words,
    calculate_uppercase_ratio,
    preprocess_text
)

np.random.seed(RANDOM_SEED)
print("✓ Librerías y módulos cargados")

✓ Librerías y módulos cargados


## 2. Cargar Datos Crudos

In [2]:
# Cargar datos usando src
# nrows=None para cargar todo, nrows=100000 para pruebas
df_train = load_raw_training_data(nrows=None)
df_test = load_raw_test_data()

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")
print(f"\nClases en train: {df_train['polarity'].unique()}")
print(f"Clases en test: {df_test['polarity'].unique()}")

✓ Dataset de entrenamiento cargado: 1600000 filas
✓ Dataset de test cargado: 498 filas
Train: (1600000, 6)
Test: (498, 6)

Clases en train: [0 4]
Clases en test: [4 0 2]


In [3]:
## 2.1. ELIMINAR DATA LEAKAGE: Remover tweets de test que están en train

print("=== VERIFICACIÓN Y LIMPIEZA DE DATA LEAKAGE ===\n")

# Obtener IDs únicos de train
train_ids = set(df_train['id'].unique())
print(f"📊 IDs únicos en train: {len(train_ids):,}")

# Obtener IDs únicos de test
test_ids_original = set(df_test['id'].unique())
print(f"📊 IDs únicos en test (original): {len(test_ids_original):,}")

# Detectar duplicados
overlap_ids = train_ids.intersection(test_ids_original)
print(f"\n🔍 IDs duplicados entre train y test: {len(overlap_ids):,}")

if len(overlap_ids) > 0:
    # Eliminar tweets de test que están en train
    df_test = df_test[~df_test['id'].isin(train_ids)].reset_index(drop=True)
    print(f"\n✅ Data leakage eliminado:")
    print(f"   - Tweets removidos: {len(overlap_ids):,}")
    print(f"   - Test después de limpieza: {len(df_test):,}")
else:
    print("\n✅ No hay data leakage")

print(f"\n📊 Shapes finales:")
print(f"   Train: {df_train.shape}")
print(f"   Test: {df_test.shape}")

=== VERIFICACIÓN Y LIMPIEZA DE DATA LEAKAGE ===

📊 IDs únicos en train: 1,598,315
📊 IDs únicos en test (original): 498

🔍 IDs duplicados entre train y test: 0

✅ No hay data leakage

📊 Shapes finales:
   Train: (1600000, 6)
   Test: (498, 6)


## 3. Implementar Función de Preprocesamiento

Esta función implementa las 9 decisiones del EDA:

In [4]:
# Usaremos la función clean_text del módulo src.preprocessing
# Esta función implementa la limpieza estándar definida en el proyecto.

def preprocess_tweet(text):
    return clean_text(text)

# Test
test_text = "@user This is SOOO COOL!!! #awesome http://example.com"
print(f"Original: {test_text}")
print(f"Procesado: {preprocess_tweet(test_text)}")

Original: @user This is SOOO COOL!!! #awesome http://example.com
Procesado: this is sooo cool awesome


## 4. Crear Features Numéricas 

In [ ]:
def extract_features(text):
    """
    Extrae features numéricas usando funciones auxiliares de src.
    """
    if pd.isna(text):
        return {
            'length': 0, 'num_words': 0, 'num_hashtags': 0, 
            'num_mentions': 0, 'num_urls': 0, 'num_uppercase': 0, 'pct_uppercase': 0.0,
            'num_intensified': 0
        }
    
    return {
        'length': len(text),
        'num_words': len(text.split()),
        'num_hashtags': len(extract_hashtags(text)),
        'num_mentions': len(extract_mentions(text)),
        'num_urls': len(extract_urls(text)),
        'num_uppercase': sum(1 for c in text if c.isupper()),
        'pct_uppercase': calculate_uppercase_ratio(text) * 100,
        'num_intensified': detect_intensified_words(text)  
    }

# Test
test_text = "@user This is SOOO COOL!!! #awesome http://example.com"
print(extract_features(test_text))

{'length': 54, 'num_words': 7, 'num_hashtags': 1, 'num_mentions': 1, 'num_urls': 1, 'num_uppercase': 9, 'pct_uppercase': 23.076923076923077, 'num_intensified': 1}


## 5. Aplicar Preprocesamiento a TRAIN

In [6]:
print("Procesando dataset TRAIN...")

# Extraer features numéricas ANTES de limpiar
tqdm.pandas(desc="Extrayendo features")
features_train = df_train['text'].progress_apply(extract_features)
features_train_df = pd.DataFrame(features_train.tolist())

# Limpiar texto
tqdm.pandas(desc="Limpiando texto")
df_train['text_clean'] = df_train['text'].progress_apply(preprocess_tweet)

# Combinar todo
df_train_processed = pd.concat([
    df_train[['polarity', 'text', 'text_clean']],
    features_train_df
], axis=1)

# Eliminar textos vacíos después de limpieza
df_train_processed = df_train_processed[df_train_processed['text_clean'].str.len() > 0]

print(f"\n✓ Train procesado: {df_train_processed.shape}")
print(f"Columnas: {list(df_train_processed.columns)}")
print(f"\nEjemplo:")
print(df_train_processed[['text', 'text_clean', 'length', 'num_hashtags']].head(3))

Procesando dataset TRAIN...


Limpiando texto: 100%|██████████| 1600000/1600000 [00:18<00:00, 87667.13it/s]



✓ Train procesado: (1596781, 11)
Columnas: ['polarity', 'text', 'text_clean', 'length', 'num_words', 'num_hashtags', 'num_mentions', 'num_urls', 'num_uppercase', 'pct_uppercase', 'num_intensified']

Ejemplo:
                                                text  \
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1  is upset that he can't update his Facebook by ...   
2  @Kenichan I dived many times for the ball. Man...   

                                          text_clean  length  num_hashtags  
0  a thats a bummer you shoulda got david carr of...     115             0  
1  is upset that he cant update his facebook by t...     111             0  
2  i dived many times for the ball managed to sav...      89             0  


## 6. Aplicar Preprocesamiento a TEST

### ⚠️ Decisión Crítica: Eliminación de Clase Neutral

**Contexto:**
- El dataset de **training NO contiene clase neutral** (solo 0=negativo y 4=positivo)
- El dataset de **test SÍ contiene 139 tweets neutrales** (clase 2), que representan el 28% del test

**Justificación para eliminar neutrales del test:**

1. **Consistencia metodológica**: Un modelo entrenado solo con clases [0, 4] no puede aprender a predecir la clase [2] que nunca vio.

2. **Imposibilidad matemática**: Pedirle al modelo que clasifique neutrales sería como entrenar para distinguir perros/gatos y luego evaluar con pájaros.

3. **Evaluación honesta**: Al evaluar solo 0 vs 4, medimos la verdadera capacidad del modelo en la tarea binaria para la que fue entrenado.

4. **Subjetividad de neutralidad**: La clase neutral es altamente subjetiva en el etiquetado manual (ej: "Going to the store" puede ser neutral o positivo según contexto).

5. **Aplicación real**: Muchos sistemas de análisis de sentimiento en producción son binarios (positivo/negativo).

**Consecuencia:**
- **Test original**: 498 tweets (182 pos, 139 neutral, 177 neg)
- **Test efectivo**: 359 tweets (182 pos, 177 neg)
- **Balance final**: 50.7% positivos / 49.3% negativos ✓ (casi perfecto)

**Métricas a evaluar** (solo sobre clases 0 y 4):
- Accuracy, Precision, Recall, F1-Score

In [7]:
print("Procesando dataset TEST...")

# DECISIÓN 9: Remover clase neutral (2) del test
df_test = df_test[df_test['polarity'].isin([0, 4])]
print(f"Test después de remover neutrales: {df_test.shape}")

# Extraer features numéricas
tqdm.pandas(desc="Extrayendo features")
features_test = df_test['text'].progress_apply(extract_features)
features_test_df = pd.DataFrame(features_test.tolist())

# Limpiar texto
tqdm.pandas(desc="Limpiando texto")
df_test['text_clean'] = df_test['text'].progress_apply(preprocess_tweet)

# Combinar todo
df_test_processed = pd.concat([
    df_test[['polarity', 'text', 'text_clean']],
    features_test_df
], axis=1)

# Eliminar textos vacíos
df_test_processed = df_test_processed[df_test_processed['text_clean'].str.len() > 0]

print(f"\n✓ Test procesado: {df_test_processed.shape}")
print(f"Clases finales: {df_test_processed['polarity'].value_counts()}")

Procesando dataset TEST...
Test después de remover neutrales: (359, 6)


Limpiando texto: 100%|██████████| 359/359 [00:00<00:00, 69172.87it/s]


✓ Test procesado: (359, 11)
Clases finales: polarity
4.0    182
0.0    177
Name: count, dtype: int64


## 7. Verificar Calidad del Preprocesamiento

In [8]:
# Comparar antes/después
print("=== COMPARACIÓN ANTES/DESPUÉS ===")
print("\nEjemplos TRAIN:")
for i in range(5):
    print(f"\nOriginal: {df_train_processed.iloc[i]['text'][:80]}")
    print(f"Limpio:   {df_train_processed.iloc[i]['text_clean'][:80]}")
    print(f"Features: length={df_train_processed.iloc[i]['length']}, "
          f"hashtags={df_train_processed.iloc[i]['num_hashtags']}, "
          f"mentions={df_train_processed.iloc[i]['num_mentions']}")

=== COMPARACIÓN ANTES/DESPUÉS ===

Ejemplos TRAIN:

Original: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got D
Limpio:   a thats a bummer you shoulda got david carr of third day to do it d
Features: length=115, hashtags=0, mentions=1

Original: is upset that he can't update his Facebook by texting it... and might cry as a r
Limpio:   is upset that he cant update his facebook by texting it and might cry as a resul
Features: length=111, hashtags=0, mentions=0

Original: @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out 
Limpio:   i dived many times for the ball managed to save the rest go out of bounds
Features: length=89, hashtags=0, mentions=1

Original: my whole body feels itchy and like its on fire 
Limpio:   my whole body feels itchy and like its on fire
Features: length=47, hashtags=0, mentions=0

Original: @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I
Limpio:   no its not behaving at a

In [9]:
# Estadísticas de limpieza
print("\n=== ESTADÍSTICAS DE LIMPIEZA ===")
print(f"\nTRAIN:")
print(f"  - Longitud promedio original: {df_train_processed['length'].mean():.1f}")
print(f"  - Longitud promedio limpia: {df_train_processed['text_clean'].str.len().mean():.1f}")
print(f"  - Tweets con hashtags: {(df_train_processed['num_hashtags'] > 0).sum()}")
print(f"  - Tweets con mentions: {(df_train_processed['num_mentions'] > 0).sum()}")
print(f"  - Tweets con URLs: {(df_train_processed['num_urls'] > 0).sum()}")

print(f"\nTEST:")
print(f"  - Longitud promedio original: {df_test_processed['length'].mean():.1f}")
print(f"  - Longitud promedio limpia: {df_test_processed['text_clean'].str.len().mean():.1f}")
print(f"  - Tweets con hashtags: {(df_test_processed['num_hashtags'] > 0).sum()}")
print(f"  - Tweets con mentions: {(df_test_processed['num_mentions'] > 0).sum()}")
print(f"  - Tweets con URLs: {(df_test_processed['num_urls'] > 0).sum()}")


=== ESTADÍSTICAS DE LIMPIEZA ===

TRAIN:
  - Longitud promedio original: 74.2
  - Longitud promedio limpia: 62.1
  - Tweets con hashtags: 35846
  - Tweets con mentions: 735297
  - Tweets con URLs: 80960

TEST:
  - Longitud promedio original: 84.3
  - Longitud promedio limpia: 71.2
  - Tweets con hashtags: 17
  - Tweets con mentions: 64
  - Tweets con URLs: 39


## 8. Guardar Datos Procesados

In [10]:
# Guardar usando src
save_processed_data(df_train_processed, 'train_processed.csv')
save_processed_data(df_test_processed, 'test_processed.csv')

print(f"\nTRAIN: {df_train_processed.shape}")
print(f"TEST: {df_test_processed.shape}")

✓ Datos guardados en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\train_processed.csv
✓ Datos guardados en: d:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\processed\test_processed.csv

TRAIN: (1596781, 11)
TEST: (359, 11)


In [12]:
## 9. Validación Final: Verificar Calidad de Datos Procesados

print("=== VALIDACIÓN DE COHERENCIA ===\n")

# 1. Verificar integridad de procesamiento
print("✅ Integridad de datos:")
print(f"   Train procesado: {len(df_train_processed):,} tweets")
print(f"   Test procesado: {len(df_test_processed):,} tweets")

# 2. Verificar distribución de clases
print(f"\n✅ Distribución TRAIN:")
print(df_train_processed['polarity'].value_counts())
print(f"   Balance: {df_train_processed['polarity'].value_counts()[0] / len(df_train_processed) * 100:.1f}% neg")

print(f"\n✅ Distribución TEST:")
print(df_test_processed['polarity'].value_counts())

# 3. Verificar que NO hay textos vacíos
empty_train = (df_train_processed['text_clean'].str.len() == 0).sum()
empty_test = (df_test_processed['text_clean'].str.len() == 0).sum()

if empty_train > 0 or empty_test > 0:
    print(f"\n❌ ERROR: Hay {empty_train} textos vacíos en TRAIN y {empty_test} en TEST")
else:
    print("\n✅ No hay textos vacíos")

# 4. Verificar features numéricas (no pueden ser todas 0)
numeric_cols = ['length', 'num_words', 'num_hashtags', 'num_mentions', 
                'num_urls', 'num_uppercase', 'pct_uppercase', 'num_intensified']

print("\n=== RANGO DE FEATURES NUMÉRICAS ===")
for col in numeric_cols:
    train_range = (df_train_processed[col].min(), df_train_processed[col].max())
    test_range = (df_test_processed[col].min(), df_test_processed[col].max())
    print(f"{col:20s} | Train: {train_range} | Test: {test_range}")

print("\n✅ Validación completa. Listo para vectorización.")

=== VALIDACIÓN DE COHERENCIA ===

✅ Integridad de datos:
   Train procesado: 1,596,781 tweets
   Test procesado: 359 tweets

✅ Distribución TRAIN:
polarity
4    798398
0    798383
Name: count, dtype: int64
   Balance: 50.0% neg

✅ Distribución TEST:
polarity
4.0    182
0.0    177
Name: count, dtype: int64

✅ No hay textos vacíos

=== RANGO DE FEATURES NUMÉRICAS ===
length               | Train: (np.int64(6), np.int64(374)) | Test: (np.float64(12.0), np.float64(144.0))
num_words            | Train: (np.int64(1), np.int64(64)) | Test: (np.float64(2.0), np.float64(30.0))
num_hashtags         | Train: (np.int64(0), np.int64(24)) | Test: (np.float64(0.0), np.float64(3.0))
num_mentions         | Train: (np.int64(0), np.int64(12)) | Test: (np.float64(0.0), np.float64(4.0))
num_urls             | Train: (np.int64(0), np.int64(5)) | Test: (np.float64(0.0), np.float64(1.0))
num_uppercase        | Train: (np.int64(0), np.int64(131)) | Test: (np.float64(0.0), np.float64(108.0))
pct_uppercase      

## 9. ¿Por Qué Este Preprocesamiento?

### El Problema: Texto ≠ Números

Los modelos de Machine Learning **no pueden procesar texto directamente**. Necesitan números.

**Ejemplo:**
```
Tweet original: "@user I LOOOVE this!!! #awesome http://example.com"
                ↓
Tweet limpio:   "i love this awesome"
                ↓
Vector TF-IDF:  [0.0, 0.52, 0.0, 0.61, ...]
```

**¿Por qué limpiamos?**  
Para que "I LOVE this" y "i love this" sean **la misma palabra** para el modelo.


### Decisiones Basadas en el EDA

**1. URLs eliminadas → Feature creada**
- **Razón**: Solo 5.1% de tweets tienen URLs y no aportan sentimiento
- **Acción**: Eliminadas del texto + conteo en `num_urls`

**2. Mentions eliminadas → Feature creada**
- **Razón**: 46.2% de tweets tienen mentions, pero son NOMBRES, no sentimiento
- **Acción**: Eliminadas del texto + conteo en `num_mentions`

**3. Hashtags convertidos a texto**
- **Razón**: 2.2% de presencia pero son DISCRIMINATIVOS (#fail vs #awesome)
- **Acción**: #awesome → awesome (extraemos el texto)

**4. Mayúsculas normalizadas**
- **Razón**: UPPERCASE indica INTENSIDAD, no polaridad
- **Acción**: Todo a lowercase + ratio de mayúsculas guardado

**5. Caracteres repetidos normalizados**
- **Razón**: "goooood" y "good" deben ser LA MISMA palabra
- **Acción**: Normalización de repeticiones excesivas

**6. Clase neutral eliminada del test**
- **Razón**: Es un problema BINARIO (positivo vs negativo)
- **Acción**: Solo clases 0 y 4 conservadas

### ¿Qué NO Hicimos (y Por Qué)?

❌ **NO eliminamos duplicados**: Son expresiones naturales comunes (retweets)
❌ **NO eliminamos stopwords aún**: Se hará en vectorización con TF-IDF
❌ **NO lemmatizamos**: Se evaluará en el próximo notebook
❌ **NO usamos features temporales**: Dataset de 2009, no generalizaría

## 10. Resumen de lo Realizado

### ✅ Decisiones Implementadas:

1. **URLs eliminadas**: Todos los http/www removidos
2. **Mentions eliminadas**: Todos los @usuario removidos
3. **Hashtags procesados**: #fail → fail (texto extraído)
4. **Mayúsculas normalizadas**: Todo a lowercase + feature `num_uppercase` y `pct_uppercase`
5. **Caracteres repetidos normalizados**: goooood → good
6. **8 Features numéricas extraídas**:
   - `length`: longitud original
   - `num_words`: cantidad de palabras
   - `num_hashtags`: cantidad de hashtags
   - `num_mentions`: cantidad de mentions
   - `num_urls`: cantidad de URLs
   - `num_uppercase`: cantidad de mayúsculas
   - `pct_uppercase`: porcentaje de mayúsculas
   - `num_intensified`: palabras con caracteres repetidos (goood, noooo)
7. **Clase neutral removida del test**: Solo 0 (neg) y 4 (pos)
8. **Textos vacíos eliminados**: Tweets sin contenido después de limpieza

